# Alpaca-py options trading basic

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/alpacahq/alpaca-py/blob/master/examples/options-trading-basic.ipynb)

- This notebook shows how to use alpaca-py with options trading API endpoints
- Please use ``paper account``. Please ``DO NOT`` use this notebook with live account. In this notebook, we place orders for options as an example.

In [2]:
# Please change the following to your own PAPER api key and secret
# You can get them from https://alpaca.markets/
# keys is stored in config.py

import config

api_key = config.ALPACA_API_KEY_PAPER
secret_key = config.ALPACA_API_SECRET_KEY_PAPER


#### We use paper environment for this example ####
paper = True # Please do not modify this. This example is for paper trading only.
####

# Below are the variables for development this documents
# Please do not change these variables

trade_api_url = None
trade_api_wss = None
data_api_url = None
option_stream_data_wss = None

In [3]:
! python3 -m install alpaca-py
! python3 -m pip install alpaca-py

In [4]:
import json
from datetime import datetime, timedelta
from zoneinfo import ZoneInfo

import alpaca
from alpaca.trading.client import TradingClient
from alpaca.data.timeframe import TimeFrame, TimeFrameUnit
from alpaca.data.historical.option import OptionHistoricalDataClient
from alpaca.trading.stream import TradingStream
from alpaca.data.live.option import OptionDataStream

from alpaca.data.requests import (
    OptionBarsRequest,
    OptionTradesRequest,
    OptionLatestQuoteRequest,
    OptionLatestTradeRequest,
    OptionSnapshotRequest,
    OptionChainRequest    
)
from alpaca.trading.requests import (
    GetOptionContractsRequest,
    GetAssetsRequest,
    MarketOrderRequest,
    GetOrdersRequest,
    ClosePositionRequest
)
from alpaca.trading.enums import (
    AssetStatus,
    ExerciseStyle,
    OrderSide,
    OrderType,
    TimeInForce,
    QueryOrderStatus 
)
from alpaca.common.exceptions import APIError

In [5]:
# to run async code in jupyter notebook
import nest_asyncio
nest_asyncio.apply()

In [6]:
# check version of alpaca-py
alpaca.__version__

'0.30.1'

# Trading Client

In [7]:
# setup clients
# - TradingClient is a class. You can instantiate it given parameters such as API key and secret key.
trade_client = TradingClient(api_key=api_key, secret_key=secret_key, paper=paper, url_override=trade_api_url)

In [8]:
# check trading account
# There are trhee new columns in the account object:
# - options_buying_power
# - options_approved_level
# - options_trading_level
acct = trade_client.get_account()
acct

{   'account_blocked': False,
    'account_number': 'PA3P9DF5CZ5G',
    'accrued_fees': '0',
    'buying_power': '199474.12',
    'cash': '99449.9',
    'created_at': datetime.datetime(2024, 5, 15, 18, 59, 39, 40242, tzinfo=TzInfo(UTC)),
    'crypto_status': <AccountStatus.ACTIVE: 'ACTIVE'>,
    'currency': 'USD',
    'daytrade_count': 0,
    'daytrading_buying_power': '0',
    'equity': '100024.22',
    'id': UUID('8124b3d1-e06e-4212-8930-0f9b5f69c050'),
    'initial_margin': '287.16',
    'last_equity': '100025.51',
    'last_maintenance_margin': '172.68',
    'long_market_value': '574.32',
    'maintenance_margin': '172.29',
    'multiplier': '2',
    'non_marginable_buying_power': '99449.9',
    'options_approved_level': 2,
    'options_buying_power': '99851.92',
    'options_trading_level': 2,
    'pattern_day_trader': False,
    'pending_transfer_in': None,
    'pending_transfer_out': None,
    'portfolio_value': '100024.22',
    'regt_buying_power': '199474.12',
    'short_marke

In [9]:
# check account configuration
# - we have new field `max_options_trading_level`
acct_config = trade_client.get_account_configurations()
acct_config

{   'dtbp_check': <DTBPCheck.ENTRY: 'entry'>,
    'fractional_trading': True,
    'max_margin_multiplier': '4',
    'max_options_trading_level': None,
    'no_shorting': False,
    'pdt_check': <PDTCheck.ENTRY: 'entry'>,
    'ptp_no_exception_entry': False,
    'suspend_trade': False,
    'trade_confirm_email': <TradeConfirmationEmail.ALL: 'all'>}

In [10]:
# get list of assets which are options enabled
# - we can filter assets by `options_enabled` attribute
# - asset object has `options_enabled` attribute if it is options enabled
req = GetAssetsRequest(
  attributes = "options_enabled"  
)
assets = trade_client.get_all_assets(req)
assets[:2]

[{   'asset_class': <AssetClass.US_EQUITY: 'us_equity'>,
     'attributes': ['has_options'],
     'easy_to_borrow': False,
     'exchange': <AssetExchange.OTC: 'OTC'>,
     'fractionable': False,
     'id': UUID('44d431bd-187a-4347-9b05-0d7d1f0ded31'),
     'maintenance_margin_requirement': 100.0,
     'marginable': False,
     'min_order_size': None,
     'min_trade_increment': None,
     'name': 'EFFECTOR THERAPEUTICS INC Common Stock',
     'price_increment': None,
     'shortable': False,
     'status': <AssetStatus.ACTIVE: 'active'>,
     'symbol': 'EFTR',
     'tradable': False},
 {   'asset_class': <AssetClass.US_EQUITY: 'us_equity'>,
     'attributes': ['fractional_eh_enabled', 'has_options'],
     'easy_to_borrow': False,
     'exchange': <AssetExchange.OTC: 'OTC'>,
     'fractionable': True,
     'id': UUID('ef8c07a7-4577-44c1-8c44-2207b69559e5'),
     'maintenance_margin_requirement': 100.0,
     'marginable': False,
     'min_order_size': None,
     'min_trade_increment': N

In [11]:
# get list of options contracts for the given underlying symbol (e.g. SPY,AAPL)
# - get_option_contracts() is a new method to get list of options contracts
# - in this example, we get 2 options contracts for SPY,AAPL
# - you can continue to fetch options contracts by specifying page_token from next_page_token of response
underlying_symbols = ["SPY", "AAPL"]
# - GetOptionContractsRequest is a request object (request model) to get list of options contracts
req = GetOptionContractsRequest(
    underlying_symbols = underlying_symbols,               # specify underlying symbols
    status = AssetStatus.ACTIVE,                           # specify asset status: active (default)
    expiration_date = None,                                # specify expiration date (specified date + 1 day range)
    expiration_date_gte = None,                            # we can pass date object
    expiration_date_lte = None,                            # or string (YYYY-MM-DD)
    root_symbol = None,                                    # specify root symbol
    type = None,                                           # specify option type (ContractType.CALL or ContractType.PUT)
    style = None,                                          # specify option style (ContractStyle.AMERICAN or ContractStyle.EUROPEAN)
    strike_price_gte = None,                               # specify strike price range
    strike_price_lte = None,                               # specify strike price range
    limit = 2,                                             # specify limit
    page_token = None,                                     # specify page token
)
res = trade_client.get_option_contracts(req)
res

{   'next_page_token': 'Mg==',
    'option_contracts': [   {   'close_price': '128.55',
                                'close_price_date': datetime.date(2024, 9, 24),
                                'expiration_date': datetime.date(2024, 9, 27),
                                'id': '92fbf37f-8a59-47b5-bc65-58a9a01f1c71',
                                'name': 'AAPL Sep 27 2024 100 Call',
                                'open_interest': '5',
                                'open_interest_date': datetime.date(2024, 9, 23),
                                'root_symbol': 'AAPL',
                                'size': '100',
                                'status': <AssetStatus.ACTIVE: 'active'>,
                                'strike_price': 100.0,
                                'style': <ExerciseStyle.AMERICAN: 'american'>,
                                'symbol': 'AAPL240927C00100000',
                                'tradable': True,
                                'type': <Cont

In [12]:
# continue to fetch option contracts if there is next_page_token in response
if res.next_page_token is not None:
    req = GetOptionContractsRequest(
        underlying_symbols = underlying_symbols,               # specify underlying symbols
        status = AssetStatus.ACTIVE,                           # specify asset status: active (default)
        expiration_date = None,                                # specify expiration date (specified date + 1 day range)
        expiration_date_gte = None,                            # we can pass date object
        expiration_date_lte = None,                            # or string (YYYY-MM-DD)
        root_symbol = None,                                    # specify root symbol
        type = None,                                           # specify option type (ContractType.CALL or ContractType.PUT) Default is "call"
        style = None,                                          # specify option style (ContractStyle.AMERICAN or ContractStyle.EUROPEAN)
        strike_price_gte = None,                               # specify strike price range
        strike_price_lte = None,                               # specify strike price range
        limit = 2,                                             # specify limit
        page_token = res.next_page_token,                      # specify page token
    )
    res = trade_client.get_option_contracts(req)
    display(res)

{   'next_page_token': 'NA==',
    'option_contracts': [   {   'close_price': '117.33',
                                'close_price_date': datetime.date(2024, 9, 24),
                                'expiration_date': datetime.date(2024, 9, 27),
                                'id': 'ef897b31-9490-4eff-883f-90eecf0aa88f',
                                'name': 'AAPL Sep 27 2024 110 Call',
                                'open_interest': '10',
                                'open_interest_date': datetime.date(2024, 9, 23),
                                'root_symbol': 'AAPL',
                                'size': '100',
                                'status': <AssetStatus.ACTIVE: 'active'>,
                                'strike_price': 110.0,
                                'style': <ExerciseStyle.AMERICAN: 'american'>,
                                'symbol': 'AAPL240927C00110000',
                                'tradable': True,
                                'type': <Con

In [13]:
# get options contract by symbol
# - get_option_contract() is a new method to get options contract by symbol or id
# - in this example, we get the first options contract from the previous response
symbol = res.option_contracts[0].symbol
contract = trade_client.get_option_contract(symbol)
contract

{   'close_price': '117.33',
    'close_price_date': datetime.date(2024, 9, 24),
    'expiration_date': datetime.date(2024, 9, 27),
    'id': 'ef897b31-9490-4eff-883f-90eecf0aa88f',
    'name': 'AAPL Sep 27 2024 110 Call',
    'open_interest': '10',
    'open_interest_date': datetime.date(2024, 9, 23),
    'root_symbol': 'AAPL',
    'size': '100',
    'status': <AssetStatus.ACTIVE: 'active'>,
    'strike_price': 110.0,
    'style': <ExerciseStyle.AMERICAN: 'american'>,
    'symbol': 'AAPL240927C00110000',
    'tradable': True,
    'type': <ContractType.CALL: 'call'>,
    'underlying_asset_id': UUID('b0b6dd9d-8b9b-48a9-ba46-b9d54906e415'),
    'underlying_symbol': 'AAPL'}

In [14]:
# get options contract by id
# - in this example, we get the first options contract from the previous response
id = res.option_contracts[0].id
contract = trade_client.get_option_contract(symbol_or_id=id)
contract

{   'close_price': '117.33',
    'close_price_date': datetime.date(2024, 9, 24),
    'expiration_date': datetime.date(2024, 9, 27),
    'id': 'ef897b31-9490-4eff-883f-90eecf0aa88f',
    'name': 'AAPL Sep 27 2024 110 Call',
    'open_interest': '10',
    'open_interest_date': datetime.date(2024, 9, 23),
    'root_symbol': 'AAPL',
    'size': '100',
    'status': <AssetStatus.ACTIVE: 'active'>,
    'strike_price': 110.0,
    'style': <ExerciseStyle.AMERICAN: 'american'>,
    'symbol': 'AAPL240927C00110000',
    'tradable': True,
    'type': <ContractType.CALL: 'call'>,
    'underlying_asset_id': UUID('b0b6dd9d-8b9b-48a9-ba46-b9d54906e415'),
    'underlying_symbol': 'AAPL'}

In [15]:
# get put options contracts
# - in this example, we get put options contracts for SPY = SPDR S&P 500 ETF
underlying_symbols = ["SPY"]

# specify expiration date range
now = datetime.now(tz = ZoneInfo("America/New_York"))
day1 = now + timedelta(days = 1)
day60 = now + timedelta(days = 60)

req = GetOptionContractsRequest(
    underlying_symbols = underlying_symbols,                     # specify underlying symbols
    status = AssetStatus.ACTIVE,                                 # specify asset status: active (default)
    expiration_date = None,                                      # specify expiration date (specified date + 1 day range)
    expiration_date_gte = day1.date(),                           # we can pass date object
    expiration_date_lte = day60.strftime(format = "%Y-%m-%d"),   # or string
    root_symbol = None,                                          # specify root symbol
    type = "put",                                                # specify option type: put
    style = ExerciseStyle.AMERICAN,                              # specify option style: american
    strike_price_gte = None,                                     # specify strike price range
    strike_price_lte = None,                                     # specify strike price range
    limit = 100,                                                 # specify limit
    page_token = None,                                           # specify page
)
res = trade_client.get_option_contracts(req)
res.option_contracts[:5]

[{   'close_price': None,
     'close_price_date': None,
     'expiration_date': datetime.date(2024, 9, 26),
     'id': 'beb77cae-f089-46ae-a865-7134408f5062',
     'name': 'SPY Sep 26 2024 300 Put',
     'open_interest': None,
     'open_interest_date': None,
     'root_symbol': 'SPY',
     'size': '100',
     'status': <AssetStatus.ACTIVE: 'active'>,
     'strike_price': 300.0,
     'style': <ExerciseStyle.AMERICAN: 'american'>,
     'symbol': 'SPY240926P00300000',
     'tradable': True,
     'type': <ContractType.PUT: 'put'>,
     'underlying_asset_id': UUID('b28f4066-5c6d-479b-a2af-85dc1a8f16fb'),
     'underlying_symbol': 'SPY'},
 {   'close_price': None,
     'close_price_date': None,
     'expiration_date': datetime.date(2024, 9, 26),
     'id': '06b21fec-64ad-43ea-b9a9-0428f12d221d',
     'name': 'SPY Sep 26 2024 305 Put',
     'open_interest': None,
     'open_interest_date': None,
     'root_symbol': 'SPY',
     'size': '100',
     'status': <AssetStatus.ACTIVE: 'active'>,
  

In [16]:
# get high open_interest contract
open_interest = 0
high_open_interest_contract = None
for contract in res.option_contracts:
    if (contract.open_interest is not None) and (int(contract.open_interest) > open_interest):
        open_interest = int(contract.open_interest)
        high_open_interest_contract = contract
high_open_interest_contract

{   'close_price': '0.03',
    'close_price_date': datetime.date(2024, 9, 24),
    'expiration_date': datetime.date(2024, 9, 26),
    'id': '6d5dcec7-6885-4dac-94cf-8a966f956182',
    'name': 'SPY Sep 26 2024 550 Put',
    'open_interest': '918',
    'open_interest_date': datetime.date(2024, 9, 23),
    'root_symbol': 'SPY',
    'size': '100',
    'status': <AssetStatus.ACTIVE: 'active'>,
    'strike_price': 550.0,
    'style': <ExerciseStyle.AMERICAN: 'american'>,
    'symbol': 'SPY240926P00550000',
    'tradable': True,
    'type': <ContractType.PUT: 'put'>,
    'underlying_asset_id': UUID('b28f4066-5c6d-479b-a2af-85dc1a8f16fb'),
    'underlying_symbol': 'SPY'}

In [26]:
# place buy put option order
# - we can place buy put option order same as buy stock/crypto order
req = MarketOrderRequest(
    symbol = high_open_interest_contract.symbol,
    qty = 10,
    side = OrderSide.BUY,
    type = OrderType.MARKET,
    time_in_force = TimeInForce.DAY,
)
res = trade_client.submit_order(req)
res

{   'asset_class': <AssetClass.US_OPTION: 'us_option'>,
    'asset_id': UUID('6d5dcec7-6885-4dac-94cf-8a966f956182'),
    'canceled_at': None,
    'client_order_id': '6d45e95c-385b-4d28-8118-81647a23cbd8',
    'created_at': datetime.datetime(2024, 9, 25, 19, 6, 33, 411383, tzinfo=TzInfo(UTC)),
    'expired_at': None,
    'extended_hours': False,
    'failed_at': None,
    'filled_at': None,
    'filled_avg_price': None,
    'filled_qty': '0',
    'hwm': None,
    'id': UUID('eb55c698-54f0-4b3f-a7ad-3c22c0ab5071'),
    'legs': None,
    'limit_price': None,
    'notional': None,
    'order_class': <OrderClass.SIMPLE: 'simple'>,
    'order_type': <OrderType.MARKET: 'market'>,
    'qty': '10',
    'replaced_at': None,
    'replaced_by': None,
    'replaces': None,
    'side': <OrderSide.BUY: 'buy'>,
    'status': <OrderStatus.PENDING_NEW: 'pending_new'>,
    'stop_price': None,
    'submitted_at': datetime.datetime(2024, 9, 25, 19, 6, 33, 403470, tzinfo=TzInfo(UTC)),
    'symbol': 'SPY240

In [27]:
# get list of orders by specifying option contract symbol
req = GetOrdersRequest(
    status = QueryOrderStatus.ALL,
    symbols = [high_open_interest_contract.symbol],
    limit = 2,
)
orders = trade_client.get_orders(req)
orders

[{   'asset_class': <AssetClass.US_OPTION: 'us_option'>,
     'asset_id': UUID('6d5dcec7-6885-4dac-94cf-8a966f956182'),
     'canceled_at': None,
     'client_order_id': '6d45e95c-385b-4d28-8118-81647a23cbd8',
     'created_at': datetime.datetime(2024, 9, 25, 19, 6, 33, 411383, tzinfo=TzInfo(UTC)),
     'expired_at': None,
     'extended_hours': False,
     'failed_at': None,
     'filled_at': datetime.datetime(2024, 9, 25, 19, 6, 33, 436420, tzinfo=TzInfo(UTC)),
     'filled_avg_price': '0.02',
     'filled_qty': '10',
     'hwm': None,
     'id': UUID('eb55c698-54f0-4b3f-a7ad-3c22c0ab5071'),
     'legs': None,
     'limit_price': None,
     'notional': None,
     'order_class': <OrderClass.SIMPLE: 'simple'>,
     'order_type': <OrderType.MARKET: 'market'>,
     'qty': '10',
     'replaced_at': None,
     'replaced_by': None,
     'replaces': None,
     'side': <OrderSide.BUY: 'buy'>,
     'status': <OrderStatus.FILLED: 'filled'>,
     'stop_price': None,
     'submitted_at': datetime

In [28]:
# below cells should be done after market open otherwise there is no position for the option contract

# get positions filtered by option contract symbol
# if you do this example outside of market hours, you will see empty list
# because we have no position in this option contract
# please wait market open and run this example again
positions = trade_client.get_all_positions()
[pos for pos in positions if pos.symbol == high_open_interest_contract.symbol]

[{   'asset_class': <AssetClass.US_OPTION: 'us_option'>,
     'asset_id': UUID('6d5dcec7-6885-4dac-94cf-8a966f956182'),
     'asset_marginable': True,
     'avg_entry_price': '0.02',
     'avg_entry_swap_rate': None,
     'change_today': '-0.3333333333333333',
     'cost_basis': '60',
     'current_price': '0.02',
     'exchange': <AssetExchange.EMPTY: ''>,
     'lastday_price': '0.03',
     'market_value': '60',
     'qty': '30',
     'qty_available': '30',
     'side': <PositionSide.LONG: 'long'>,
     'swap_rate': None,
     'symbol': 'SPY240926P00550000',
     'unrealized_intraday_pl': '0',
     'unrealized_intraday_plpc': '0',
     'unrealized_pl': '0',
     'unrealized_plpc': '0',
     'usd': None}]

In [29]:
# get positions by symbol
trade_client.get_open_position(symbol_or_asset_id=high_open_interest_contract.symbol)


{   'asset_class': <AssetClass.US_OPTION: 'us_option'>,
    'asset_id': UUID('6d5dcec7-6885-4dac-94cf-8a966f956182'),
    'asset_marginable': True,
    'avg_entry_price': '0.02',
    'avg_entry_swap_rate': None,
    'change_today': '-0.3333333333333333',
    'cost_basis': '60',
    'current_price': '0.02',
    'exchange': <AssetExchange.EMPTY: ''>,
    'lastday_price': '0.03',
    'market_value': '60',
    'qty': '30',
    'qty_available': '30',
    'side': <PositionSide.LONG: 'long'>,
    'swap_rate': None,
    'symbol': 'SPY240926P00550000',
    'unrealized_intraday_pl': '0',
    'unrealized_intraday_plpc': '0',
    'unrealized_pl': '0',
    'unrealized_plpc': '0',
    'usd': None}

In [30]:
# get positions by contract id
trade_client.get_open_position(symbol_or_asset_id = high_open_interest_contract.id)

{   'asset_class': <AssetClass.US_OPTION: 'us_option'>,
    'asset_id': UUID('6d5dcec7-6885-4dac-94cf-8a966f956182'),
    'asset_marginable': True,
    'avg_entry_price': '0.02',
    'avg_entry_swap_rate': None,
    'change_today': '-0.3333333333333333',
    'cost_basis': '60',
    'current_price': '0.02',
    'exchange': <AssetExchange.EMPTY: ''>,
    'lastday_price': '0.03',
    'market_value': '60',
    'qty': '30',
    'qty_available': '30',
    'side': <PositionSide.LONG: 'long'>,
    'swap_rate': None,
    'symbol': 'SPY240926P00550000',
    'unrealized_intraday_pl': '0',
    'unrealized_intraday_plpc': '0',
    'unrealized_pl': '0',
    'unrealized_plpc': '0',
    'usd': None}

In [32]:
# close the option position (sell the option contract)
trade_client.close_position(
    symbol_or_asset_id = high_open_interest_contract.symbol,
    close_options = ClosePositionRequest(qty = "20")
)

{   'asset_class': <AssetClass.US_OPTION: 'us_option'>,
    'asset_id': UUID('6d5dcec7-6885-4dac-94cf-8a966f956182'),
    'canceled_at': None,
    'client_order_id': 'cfe30eb8-0e48-4aac-a0c6-36bf10893274',
    'created_at': datetime.datetime(2024, 9, 25, 19, 12, 25, 985400, tzinfo=TzInfo(UTC)),
    'expired_at': None,
    'extended_hours': False,
    'failed_at': None,
    'filled_at': None,
    'filled_avg_price': None,
    'filled_qty': '0',
    'hwm': None,
    'id': UUID('31194028-6e2e-46e2-aa6a-5b462f840ea7'),
    'legs': None,
    'limit_price': None,
    'notional': None,
    'order_class': <OrderClass.SIMPLE: 'simple'>,
    'order_type': <OrderType.MARKET: 'market'>,
    'qty': '20',
    'replaced_at': None,
    'replaced_by': None,
    'replaces': None,
    'side': <OrderSide.SELL: 'sell'>,
    'status': <OrderStatus.PENDING_NEW: 'pending_new'>,
    'stop_price': None,
    'submitted_at': datetime.datetime(2024, 9, 25, 19, 12, 25, 982412, tzinfo=TzInfo(UTC)),
    'symbol': 'SP

In [ ]:
# exercise the options position
# - this method does not return anything
trade_client.exercise_options_position(
    symbol_or_contract_id = high_open_interest_contract.symbol
)

# Trade Update (Stream)

With TradingStream client, you can get updates about trades

fyi. you can open this notebook in another window and run below cell to check trade updates.

In [ ]:
# subscribe trade updates
trade_stream_client = TradingStream(api_key, secret_key, paper=paper, url_override = trade_api_wss)

async def trade_updates_handler(data):
    print(data)

trade_stream_client.subscribe_trade_updates(trade_updates_handler)
trade_stream_client.run()

# Market Data (Historical)

In [ ]:
# setup option historical data client
option_historical_data_client = OptionHistoricalDataClient(api_key, secret_key, url_override = data_api_url)

In [ ]:
# get options historical bars by symbol
req = OptionBarsRequest(
    symbol_or_symbols = high_open_interest_contract.symbol,
    timeframe = TimeFrame(amount = 1, unit = TimeFrameUnit.Hour),   # specify timeframe
    start = now - timedelta(days = 5),                              # specify start datetime, default=the beginning of the current day.
    # end_date=None,                                                # specify end datetime, default=now
    limit = 2,                                                      # specify limit
)
option_historical_data_client.get_option_bars(req).df

In [ ]:
# get options historical trades by symbol
req = OptionTradesRequest(
    symbol_or_symbols = high_open_interest_contract.symbol,
    start = now - timedelta(days = 5),                              # specify start datetime, default=the beginning of the current day.
    # end=None,                                                     # specify end datetime, default=now
    limit = 2,                                                      # specify limit
)
option_historical_data_client.get_option_trades(req).df

In [ ]:
# get options exchange codes
option_historical_data_client.get_option_exchange_codes()

In [ ]:
# get option latest quote by symbol
req = OptionLatestQuoteRequest(
    symbol_or_symbols = [high_open_interest_contract.symbol],
)
option_historical_data_client.get_option_latest_quote(req)

In [ ]:
# get option latest trade by symbol
req = OptionLatestTradeRequest(
    symbol_or_symbols = [high_open_interest_contract.symbol],
)
option_historical_data_client.get_option_latest_trade(req)

In [ ]:
# get option snapshot by symbol
req = OptionSnapshotRequest(
    symbol_or_symbols = [high_open_interest_contract.symbol],
)
option_historical_data_client.get_option_snapshot(req)

In [ ]:
# get option chain by underlying_symbol
req = OptionChainRequest(
    underlying_symbol = high_open_interest_contract.underlying_symbol,
)
option_historical_data_client.get_option_chain(req)

# Market Data (Stream)

In [ ]:
option_data_stream_client = OptionDataStream(api_key, secret_key, url_override = option_stream_data_wss)

async def option_data_stream_handler(data):
    print(data)

symbols = [
    high_open_interest_contract.symbol,
]

option_data_stream_client.subscribe_quotes(option_data_stream_handler, *symbols) 
option_data_stream_client.subscribe_trades(option_data_stream_handler, *symbols)

option_data_stream_client.run()